In [9]:
import gymnasium as gym
import torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts

In [10]:
task = 'CartPole-v0'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
# logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

In [12]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.Shme([lambda: gym.make(task) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(test_num)])

In [13]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
env = gym.make(task)
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [14]:
policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

In [15]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    )
print(f'Finished training! Use {result["duration"]}')

/home/behnamnia/.conda/envs/rl39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Epoch #1: 10001it [00:19, 510.77it/s, env_step=10000, len=140, loss=0.231, n/ep=0, n/st=10, rew=140.00]                          


Epoch #1: test_reward: 188.040000 ± 18.481840, best_reward: 188.040000 ± 18.481840 in #1


Epoch #2: 10001it [00:52, 190.30it/s, env_step=20000, len=200, loss=0.283, n/ep=0, n/st=10, rew=200.00]                           


Epoch #2: test_reward: 187.160000 ± 16.191183, best_reward: 188.040000 ± 18.481840 in #1


Epoch #3:  57%|#####7    | 5710/10000 [01:57<01:28, 48.59it/s, env_step=25710, len=197, n/ep=1, n/st=10, rew=197.00]              

Finished training! Use 197.16s


In [16]:
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/home/behnamnia/.conda/envs/rl39/lib/python3.9/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
/home/behnamnia/.conda/envs/rl39/lib/python3.9/site-packages/gymnasium/envs/classic_control/cartpole.py:213: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v0", render_mode="rgb_array")
  gym.logger.warn(


{'n/ep': 1,
 'n/st': 200,
 'rews': array([200.]),
 'lens': array([200]),
 'idxs': array([0]),
 'rew': 200.0,
 'len': 200.0,
 'rew_std': 0.0,
 'len_std': 0.0}